In [ ]:
# @title 데이터 준비
# https://drive.google.com/file/d/1z-HEvwpoTDbef_EowtGp7mUINlDRLFw7/view?usp=sharing
# https://drive.google.com/file/d/1l7bAiyAX4XSitPZgSBDXlMuZo3A7GaJd/view?usp=sharing
import gdown, zipfile, os

file_id = '1l7bAiyAX4XSitPZgSBDXlMuZo3A7GaJd'

gdown.download(f'https://drive.google.com/uc?id={file_id}', 'SUV_kor.zip', quiet=False)

dir = 'SUV-classification2'
os.makedirs(dir, exist_ok=True)  # 없으면 생성

with zipfile.ZipFile('SUV_kor.zip', 'r') as z:
    z.extractall(dir)

Downloading...
From (original): https://drive.google.com/uc?id=1l7bAiyAX4XSitPZgSBDXlMuZo3A7GaJd
From (redirected): https://drive.google.com/uc?id=1l7bAiyAX4XSitPZgSBDXlMuZo3A7GaJd&confirm=t&uuid=0be21f35-923b-4c76-af25-b09caf1d387e
To: /content/SUV_kor.zip
100%|██████████| 1.75G/1.75G [00:06<00:00, 250MB/s]


In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# 데이터 로딩 함수
def load_data(base_dir='SUV-classification2'):
    images = []
    labels = []

    base = os.path.join(base_dir, 'SUV')

    for category in ['Hyundai', 'Kia']:
        category_path = os.path.join(base, category)

        for filename in os.listdir(category_path):
            if '.jpg' in filename:
                file_path = os.path.join(category_path, filename)
                images.append(file_path)
                texts = filename.replace('_', '$').replace('-', '$')
                texts = texts.split('$')
                labels.append(texts[1])  # 차량 종류 라벨

    df_temp = pd.DataFrame({'path': images, 'label': labels})
    return df_temp

# 데이터 로드
df_temp = load_data()
data_set = pd.DataFrame()

# 각 라벨마다 1000개씩 샘플링
for g in df_temp['label'].unique():
    if len(df_temp[df_temp['label'] == g]) > 1000:
        temp_df = df_temp[df_temp['label'] == g].sample(1000)
        data_set = pd.concat([data_set, temp_df])

print(data_set['label'].value_counts())



label
Santafe     1000
Tucson      1000
Kona        1000
Veracruz    1000
Palisade    1000
Sorento     1000
Sportage    1000
Mohave      1000
Niro        1000
Seltos      1000
Soul        1000
Carens      1000
Name: count, dtype: int64


In [ ]:
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img) / 255.0  # 이미지 정규화
    return img_array

images_resized = np.array([preprocess_image(path) for path in data_set['path']])
labels = data_set['label'].values

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# 학습/검증 데이터 분리
train_images, val_images, train_labels, val_labels = train_test_split(
    images_resized, encoded_labels, test_size=0.2, random_state=42
)

print(train_images.shape, val_images.shape)


(9600, 224, 224, 3) (2400, 224, 224, 3)


In [ ]:

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# MobileNetV2 모델 불러오기
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# 기본 모델의 레이어 동결
for layer in base_model.layers:
    layer.trainable = False  # 가중치 고정

# 새로운 출력층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)  # Dropout 추가
x = Dense(1024, activation='relu', kernel_initializer='he_normal')(x)  # Dense 레이어 추가
output = Dense(12, activation='softmax')(x)  # 클래스 수는 12개

# 최종 모델 정의
model = Model(inputs=base_model.input, outputs=output)


9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# MobileNetV2 모델 불러오기
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# 기본 모델의 레이어 동결
for layer in base_model.layers:
    layer.trainable = False

# 새로운 출력층 설정
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)  # Dropout 추가
x = Dense(1024, activation='relu', kernel_initializer='he_normal')(x)
x = Dropout(0.3)(x)
output = Dense(12, activation='softmax')(x)  # 12개의 클래스

# 모델 컴파일
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=50)

# 학습 결과 시각화
import matplotlib.pyplot as plt
pd.DataFrame(history.history).plot()
plt.show()

In [ ]:
# Fine-tuning 시작 (기존 레이어의 일부만 학습)
for layer in base_model.layers[:-5]:
    layer.trainable = True

# 모델 컴파일 (learning rate 낮추기)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fine-tuning 학습
history_ft = model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=50)

# Fine-tuning 결과 시각화
pd.DataFrame(history_ft.history).plot()
plt.show()


In [ ]:
# 평가
val_loss, val_accuracy = model.evaluate(val_images, val_labels)
print(f" Accuracy: {val_accuracy:.4f},  Loss: {val_loss:.4f}, ")

# 모델 저장
model.save("MobileNet_2_model.h5")